In [1]:
import importlib
import cv2
import json
import os
import math
import copy
from _paths import nomeroff_net_dir

from nomeroff_net.pipes.number_plate_text_readers.base.ocr import OCR
from nomeroff_net.tools.mcm import modelhub

/usr/local/lib64/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib64/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib64/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib64/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# auto download latest dataset
#info = modelhub.download_dataset_for_model("EuUa1995")
#PATH_TO_DATASET = info["dataset_path"]
PATH_TO_DATASET = os.path.join(nomeroff_net_dir, "data/dataset/TextDetector/ocr_example")

In [3]:
def compare(model_path='latest',
           text_detector_name = "eu_ua_2004_2015",
           img_format = "png",
           root_dir=os.path.join(PATH_TO_DATASET, "test"),
           predicted_part_size=1,
           acc_less_than = 0.7,
           replace_tamplate = None,
           model_conf = None):
    if replace_tamplate is None:
        replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}
    if model_conf is None:
        model_conf = copy.deepcopy(modelhub.models[text_detector_name])
    text_detector = OCR(model_name=text_detector_name, **model_conf)
    text_detector.init_label_converter()
    text_detector.load(model_path)

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dir_name, subdir_list, file_list in os.walk(ann_dir):
        for fname in file_list:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    accs      = []
    N = math.ceil(len(imgs) / predicted_part_size)
    for i in range(N):
        part = i*predicted_part_size
        part_imgs = imgs[part:part+predicted_part_size]
        xs = text_detector.preprocess(part_imgs)
        predicted_part, net_out_value_part = text_detector.predict(xs, return_acc=True)
        predicted += predicted_part


        # get accuracy
        if acc_less_than >= 1:
            # not process acc
            accs  += [1 for _predicted in predicted_part]
            continue
        # process accuracy
        acc_part = []
        for _predicted, _net_out_value in zip(predicted_part, net_out_value_part):
            acc = text_detector.get_acc([_net_out_value], [_predicted])
            acc_part.append(acc)
        accs  += acc_part


    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        acc            = accs[i]
        jsonData["moderation"]["predicted"] = predicted_item

        if jsonData["description"].lower() == jsonData["moderation"]["predicted"].lower() and acc > acc_less_than:
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}' with acc {}, real: '{}' in file {}".format(
                jsonData["moderation"]["predicted"].lower(),
                acc,
                jsonData["description"].lower(),
                json_path))
            err_cnt = err_cnt+1
            jsonData["moderation"]["isModerated"] = 0
        with open(json_path, "w", encoding='utf8') as jsonWF:
            json.dump(jsonData, jsonWF,  ensure_ascii=False)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))

In [4]:
compare(model_path="latest", acc_less_than = 0.75, model_conf = {
      "letters": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H",
              "I", "K", "M", "O", "P", "T", "X", "Y", "Z"],
      "max_text_len": 9
     })

/usr/local/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


LOADED 4 ANNOTATIONS
LOADED 4 IMAGES
PREDICTED 4 IMAGES
Predicted 'ap222bi' with acc 0.8141323924064636, real: '0038sc' in file /var/www/nomeroff-net33/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/0038SC-0.json
Predicted 'bo2514kct' with acc 0.9932663440704346, real: '2914 kc-7' in file /var/www/nomeroff-net33/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/10001_2914KC7_0.json
Predicted 'b371h2o' with acc 0.9075599908828735, real: 'x371hk96' in file /var/www/nomeroff-net33/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/1000_X371HK96_0.json
Error detection count: 3
Accuracy: 0.25


/var/www/nomeroff-net33/examples/ju/dataset_tools/../../../nomeroff_net/pipes/number_plate_text_readers/base/ocr.py:247: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  pred_texts = decode_batch(torch.Tensor(net_out_value), self.label_converter)


In [5]:
# # or load your own trained model
# compare(
#     model_path="../../../data/models/shufflenet_v2_x2_0/anpr_ocr_eu_2004_2015_2022_11_11_shufflenet_v2_x2_0.ckpt",
#     acc_less_than = 0.75,
#     model_conf={
#       "backbone": "shufflenet_v2_x2_0",
#       "letters": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H",
#                   "I", "K", "M", "O", "P", "T", "X", "Y", "Z"],
#       "max_text_len": 8,
#       "height": 50,
#       "width": 200,
#       "color_channels": 3,
#       "bidirectional": True,
#       "hidden_size": 32,
#       "linear_size": 416
#     }
# )